In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [26]:
jr = pd.read_csv("DataStore/2018-07-P3/JTHReg0727/JTH_res_out_0726.csv",index_col=0)
rr = pd.read_csv("DataStore/2018-07-P2/Reg0726/reg_pairs_out_0726.csv",index_col=0)
rr["samp"] = rr["samp"].apply(eval)
rr = pd.concat([jr,rr],axis=0)
# Reset index
rr = rr.reset_index(drop=True)
rr = rr.replace("C(inv_msa_match)[T.True]", "$I(MSA \, Match)$")

In [3]:
rr.columns

Index(['1975-85', '1985-95', '1995-05', '2005-15', 'LKS', 'Model', 'Model Num',
       'index', 'samp'],
      dtype='object')

In [27]:
rr["id"] = list(zip(rr["samp"], rr["Model Num"]))
rr["id"] = rr["id"].astype(str)

# KS dict
ksd = rr[["id", "LKS"]].drop_duplicates()
ksd = dict(zip(ksd["id"], ksd["LKS"]))

# Sample dict
sd = rr[["id", "samp"]].drop_duplicates()
sd = dict(zip(sd["id"], sd["samp"]))

In [5]:
rr.head()

,1975-85,1985-95,1995-05,2005-15,LKS,Model,Model Num,index,samp,id
0,0.0530***,0.0624***,0.0646***,0.0740***,perc_match_10,"Perc Match Targ MSA, Year FE",0,"$I(MSA \, Match)$",JTH Rep,"('JTH Rep', 0)"
1,(0.0016),(0.0011),(0.0009),(0.0012),perc_match_10,"Perc Match Targ MSA, Year FE",0,NaN,JTH Rep,"('JTH Rep', 0)"
2,58647,107358,185154,154619,perc_match_10,"Perc Match Targ MSA, Year FE",0,$N$,JTH Rep,"('JTH Rep', 0)"
3,0.02,0.03,0.03,0.02,perc_match_10,"Perc Match Targ MSA, Year FE",0,Adjusted $R^2$,JTH Rep,"('JTH Rep', 0)"
4,0.0523***,0.0616***,0.0642***,0.0729***,perc_match_10,"Perc Match Targ MSA, PC FE",1,"$I(MSA \, Match)$",JTH Rep,"('JTH Rep', 1)"


### 1. Results with Year FE and PC Controls

- 4 tables: Norm DV, Norm LDA Vecs, Raw DV, Raw LDA Vecs


Samp    | Norm DV/LDA | Raw DV/LDA 
---------|---------|---------
JTH Rep | 2, 3 | 0,1 
JTH Targ Sim | 6,7 |4,5 
Naics Name | 8,10 | 0,2 
PC | 8,10 | 0,2 | 

In [69]:
repl1 = pickle.load(open("DataStore/2018-07-P3/reg_names_dict.pkl", "rb"))
repl2 = {k: "\multicolumn{2}{c}{"+v+"}" if len(v)>1 else v for k,v in repl1.items() }

In [70]:
tab_ind = {}

tab_ind["norm", "ldavecs"] = [str(i) for i in [("JTH Rep", 2),("JTH Rep", 3), ("JTH Targ Sim", 2), ("JTH Targ Sim", 3),
       (("ldavecs", "naics_name"), 8), (("ldavecs", "naics_name"), 10), (("ldavecs", "primclass"), 8),
       (("ldavecs", "primclass"), 10)]]
tab_ind["raw", "ldavecs"] = [str(i) for i in [("JTH Rep", 0),("JTH Rep", 1), ("JTH Targ Sim", 0), ("JTH Targ Sim", 1),
       (("ldavecs", "naics_name"), 0), (("ldavecs", "naics_name"), 2), (("ldavecs", "primclass"), 0),
       (("ldavecs", "primclass"), 2)]]
tab_ind["norm", "docvecs"] = [str(i) for i in [("JTH Rep", 2),("JTH Rep", 3), ("JTH Targ Sim", 6), ("JTH Targ Sim", 7),
       (("docvecs", "naics_name"), 8), (("docvecs", "naics_name"), 10), (("docvecs", "primclass"), 8),
       (("docvecs", "primclass"), 10)]]
tab_ind["raw", "docvecs"] = [str(i) for i in [("JTH Rep", 0),("JTH Rep", 1), ("JTH Targ Sim", 4), ("JTH Targ Sim", 5),
       (("docvecs", "naics_name"), 0), (("docvecs", "naics_name"), 2), (("docvecs", "primclass"), 0),
       (("docvecs", "primclass"), 2)]]    


In [71]:
v = "norm"
dm = "docvecs"
ygs = ["1975-85", "1985-95", "1995-05", "2005-15"]
tabp = {}
tabl = {}
for v,dm in tab_ind.keys():
    for yg in ygs:
        rr_t = rr.loc[rr["id"].isin(tab_ind[v,dm])]
        rg = rr_t[[yg, "index", "id"]].groupby("id")
        tab = pd.concat([g[[yg, "index"]].set_index("index") for n,g in rg], axis=1)
        
        # Add other columns
        tab.loc["Year FE"] = [True]*len(tab.columns)
        tab.loc["PC FE"] = [False, True, False, True, False, True, False, True]

        # Save index
        ind_ord = tab.index.tolist()

        # Add KS, Sample rows, Model Numbers
        KS = [str(ksd[i]) for i in rg.groups.keys()]
        KS[1::2] = [np.nan]*len(KS[1::2])
        tab.loc["Spillover"] = KS
        Sample = [str(sd[i]) for i in rg.groups.keys()]
        Sample[1::2] = [np.nan]*len(Sample[1::2])
        tab.loc["Sample"] = Sample
        tab.loc["Model"] = ["({0})".format(i+1) for i in range(len(tab.columns))]
        # Reorder
        tab = tab.reindex(["Sample", "Spillover", "Model"]+ind_ord).reset_index()
        # Create gaps
        tab = tab.fillna("")
        tab.columns = [""]*len(tab.columns)
        
        # Pandas tables
        tab1 = tab.replace(repl1)
        tabp[v,dm,yg] = tab1
        
        # Latex tables
        tab2 = tab.replace(repl2)
        tabl[v,dm,yg] = tab2

In [72]:
for k,v in tabp.items():
    print(k)
    display(v)

('norm', 'ldavecs', '1975-85')


,,,,,,,,,
0,Sample,JTH Cite,,JTH Cite Sim,,NAICS,,Primclass,
1,Spillover,"$pct\,cites\,in\,MSA_{T}$",,"$\overline{sim_{LDA}(T,j)}$",,"$sim_{LDA}(i,j)$",,"$sim_{LDA}(i,j)$",
2,,(1),(2),(3),(4),(5),(6),(7),(8)
3,"$I(MSA \, Match)$",0.2448***,0.2416***,0.0033,-0.0008,0.0465***,0.0313***,0.0566***,0.0423***
4,,(0.0074),(0.0073),(0.0139),(0.0135),(0.0050),(0.0052),(0.0057),(0.0062)
5,$N$,58647,58647,38541,38541,193173,193173,171099,171099
6,Adjusted $R^2$,0.02,0.03,0.00,0.11,0.11,0.00,0.18,0.00
7,Year FE,True,True,True,True,True,True,True,True
8,PC FE,False,True,False,True,False,True,False,True


('norm', 'ldavecs', '1985-95')


,,,,,,,,,
0,Sample,JTH Cite,,JTH Cite Sim,,NAICS,,Primclass,
1,Spillover,"$pct\,cites\,in\,MSA_{T}$",,"$\overline{sim_{LDA}(T,j)}$",,"$sim_{LDA}(i,j)$",,"$sim_{LDA}(i,j)$",
2,,(1),(2),(3),(4),(5),(6),(7),(8)
3,"$I(MSA \, Match)$",0.2881***,0.2846***,0.0280***,0.0057,0.0621***,0.0535***,0.0647***,0.0516***
4,,(0.0052),(0.0051),(0.0094),(0.0091),(0.0041),(0.0043),(0.0045),(0.0050)
5,$N$,107358,107358,69612,69612,281625,281625,252478,252478
6,Adjusted $R^2$,0.03,0.05,0.00,0.10,0.10,0.00,0.18,0.00
7,Year FE,True,True,True,True,True,True,True,True
8,PC FE,False,True,False,True,False,True,False,True


('norm', 'ldavecs', '1995-05')


...

,,,,,,,,,
0,Sample,JTH Cite,,JTH Cite Sim,,NAICS,,Primclass,
1,Spillover,"$pct\,cites\,in\,MSA_{T}$",,"$\overline{sim_{DV}(T,j)}$",,"$sim_{DV}(i,j)$",,"$sim_{DV}(i,j)$",
2,,(1),(2),(3),(4),(5),(6),(7),(8)
3,"$I(MSA \, Match)$",0.0624***,0.0616***,0.0468***,0.0469***,0.0070***,0.0082***,0.0108***,0.0111***
4,,(0.0011),(0.0011),(0.0014),(0.0014),(0.0006),(0.0006),(0.0007),(0.0007)
5,$N$,107358,107358,69612,69612,281222,281222,252174,252174
6,Adjusted $R^2$,0.03,0.05,0.02,0.05,0.00,0.07,0.00,0.07
7,Year FE,True,True,True,True,True,True,True,True
8,PC FE,False,True,False,True,False,True,False,True


('raw', 'docvecs', '1995-05')


,,,,,,,,,
0,Sample,JTH Cite,,JTH Cite Sim,,NAICS,,Primclass,
1,Spillover,"$pct\,cites\,in\,MSA_{T}$",,"$\overline{sim_{DV}(T,j)}$",,"$sim_{DV}(i,j)$",,"$sim_{DV}(i,j)$",
2,,(1),(2),(3),(4),(5),(6),(7),(8)
3,"$I(MSA \, Match)$",0.0646***,0.0642***,0.0408***,0.0422***,0.0078***,0.0082***,0.0116***,0.0109***
4,,(0.0009),(0.0009),(0.0010),(0.0010),(0.0005),(0.0005),(0.0005),(0.0005)
5,$N$,185154,185154,122217,122217,437685,437685,401623,401623
6,Adjusted $R^2$,0.03,0.05,0.02,0.06,0.00,0.08,0.00,0.08
7,Year FE,True,True,True,True,True,True,True,True
8,PC FE,False,True,False,True,False,True,False,True


('raw', 'docvecs', '2005-15')


,,,,,,,,,
0,Sample,JTH Cite,,JTH Cite Sim,,NAICS,,Primclass,
1,Spillover,"$pct\,cites\,in\,MSA_{T}$",,"$\overline{sim_{DV}(T,j)}$",,"$sim_{DV}(i,j)$",,"$sim_{DV}(i,j)$",
2,,(1),(2),(3),(4),(5),(6),(7),(8)
3,"$I(MSA \, Match)$",0.0740***,0.0729***,0.0420***,0.0423***,0.0076***,0.0077***,0.0083***,0.0079***
4,,(0.0012),(0.0012),(0.0016),(0.0015),(0.0004),(0.0004),(0.0005),(0.0004)
5,$N$,154619,154619,52710,52710,569252,569252,529686,529686
6,Adjusted $R^2$,0.02,0.05,0.02,0.05,0.00,0.06,0.00,0.07
7,Year FE,True,True,True,True,True,True,True,True
8,PC FE,False,True,False,True,False,True,False,True


In [74]:
def conv_latex(l, title, ncols = 8):
    l = ' '.join(l.split())
    l = l.replace("\\toprule \\\ \\midrule ",\
                  "\\toprule \multicolumn{"+str(ncols)+"}{c}{"+title+"} \\\ \hline ")
    l = l.replace(" & &", " & ")
    l = l.replace(" & \\\ ", " \\\ ")
    l = l.replace("(8) \\", "(8) \\\ \midrule  ")
    return l

for k,v in tabl.items():
    print("\n"+str(k)+"\n"+"\n")
    ncols = len(v.columns)
    with pd.option_context("max_colwidth", 1000):
        print(conv_latex(v.to_latex(index=False,escape=False, column_format="lcccccccc"),
                        title=k[-1],
                        ncols=ncols))


('norm', 'ldavecs', '1975-85')


\begin{tabular}{lcccccccc} \toprule \multicolumn{9}{c}{1975-85} \\ \hline Sample & \multicolumn{2}{c}{JTH Cite} &  \multicolumn{2}{c}{JTH Cite Sim} &  \multicolumn{2}{c}{NAICS} &  \multicolumn{2}{c}{Primclass} \\ Spillover & \multicolumn{2}{c}{$pct\,cites\,in\,MSA_{T}$} &  \multicolumn{2}{c}{$\overline{sim_{LDA}(T,j)}$} &  \multicolumn{2}{c}{$sim_{LDA}(i,j)$} &  \multicolumn{2}{c}{$sim_{LDA}(i,j)$} \\ & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) \\ \midrule  \ $I(MSA \, Match)$ & 0.2448*** & 0.2416*** & 0.0033 & -0.0008 & 0.0465*** & 0.0313*** & 0.0566*** & 0.0423*** \\ & (0.0074) & (0.0073) & (0.0139) & (0.0135) & (0.0050) & (0.0052) & (0.0057) & (0.0062) \\ $N$ & 58647 & 58647 & 38541 & 38541 & 193173 & 193173 & 171099 & 171099 \\ Adjusted $R^2$ & 0.02 & 0.03 & 0.00 & 0.11 & 0.11 & 0.00 & 0.18 & 0.00 \\ Year FE & True & True & True & True & True & True & True & True \\ PC FE & False & True & False & True & False & True & False & True \\ \bottomrul

### 2. NAICS Results with Field Sim

- 4 tables for each vectorization: Norm FE, Norm No FE, Raw FE, Raw No FE


Samp    | Norm FE | Norm No FE | Raw FE | Raw No FE
---------|---------|---------|---------|---------
NAICS MSA | 24 | 11 | 19 | 3
PC | 25 | 12 | 20 |4
PC Int | 27 | 14 | 22 | 6
PC MSA | 26 | 13 | 21 | 5
PC MSA Int | 28 | 15 | 23 |7

In [77]:
repl1 = pickle.load(open("DataStore/2018-07-P3/reg_names_dict.pkl", "rb"))
repl2 = {k: "\multicolumn{2}{c}{"+v+"}" if len(v)>1 else v for k,v in repl1.items() }

tab_ind = {}
# for dm in ["ldavecs", "docvecs"]:
#     nf = [24, 25, 27, 26, 28]
#     tab_ind["norm", dm, "FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
#     nf = [11,12,14,13,15]
#     tab_ind["norm", dm, "No FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf]
#     nf = [19,20,22,21,23]
#     tab_ind["raw", dm, "FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
#     nf = [3,4,6,5,7]
#     tab_ind["raw", dm, "No FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 

for dm in ["ldavecs", "docvecs"]:
    nf = [25, 27, 26, 28]
    tab_ind["norm", dm, "FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
    nf = [12,14,13,15]
    tab_ind["norm", dm, "No FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf]
    nf = [20,22,21,23]
    tab_ind["raw", dm, "FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
    nf = [4,6,5,7]
    tab_ind["raw", dm, "No FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
# Add names for standard errors
rr2 = rr.copy()
se_index = rr2[[yg, "index"]].loc[rr2["index"].isnull()].index
above_var = rr2.loc[se_index-1, "index"].tolist()
rr2.loc[se_index, "index"] = ["(s.e.) "+v for v in above_var]    

In [78]:
v = "norm"
dm = "docvecs"
fe = "FE"
ygs = ["1975-85", "1985-95", "1995-05", "2005-15"]
tabp = {}
tabl = {}
yg = ygs[1]
for v in ["norm", "raw"]:
    for dm in ["docvecs", "ldavecs"]:
        for fe in ["FE", "No FE"]:
            for yg in ygs:
                rr_t = rr2.loc[rr["id"].isin(tab_ind[v,dm,fe])]
                re_ind = rr_t[[yg, "index"]].drop_duplicates("index", keep="first")["index"].tolist()
                # Re-order re-ind
                n_ind = re_ind.index("$N$")
                re_ind = re_ind[:n_ind]+re_ind[n_ind+2:]+re_ind[n_ind:n_ind+2]

                rg = rr_t[[yg, "index", "id"]].groupby("id")

                tab = pd.concat([g[[yg, "index"]].set_index("index").reindex(re_ind) for n,g in rg], axis=1)

                # Add other columns
                tab.loc["Year FE"] = [True]*len(tab.columns)

                if fe == "FE":
#                     tab.loc["NAICS FE"] = [True, False, False, False, False]
#                     tab.loc["PC FE"] = [True, True, True, True, True]
#                     tab.loc["NAICS FE"] = [True, False, False, False, False]
                    tab.loc["PC FE"] = [True, True, True, True]
                else:
#                     tab.loc["NAICS FE"] = [False]*len(tab.columns)
                    tab.loc["PC FE"] = [False]*len(tab.columns)

                # Save index
                ind_ord = tab.index.tolist()
                tab.loc["Model"] = ["({0})".format(i+1) for i in range(len(tab.columns))]
                # Reorder
                tab = tab.reindex(["Model"]+ind_ord).reset_index()
                # Create gaps
                tab = tab.fillna("")
                # Get rid of se labels
                tab.loc[tab["index"].apply(lambda x: "(s.e.)" in x), "index"] = ""

                tabc = tab.copy()
                tab.columns = [""]*len(tab.columns)

                # Pandas tables
                tab1 = tab.replace(repl1)
                tabp[v,dm,fe,yg] = tab1

                # Latex tables
                tab2 = tab.replace(repl2)
                tabl[v,dm,fe,yg] = tab2


In [79]:
for k,v in tabp.items():
    print(k)
    display(v)

('norm', 'docvecs', 'FE', '1975-85')


,,,,,
0,,(1),(2),(3),(4)
1,"$I(MSA \, Match)$",,,,
2,,,,,
3,"$ sim_{DV}(pc_{i}, pc_{j})$",,,,
4,,,,,
5,"$sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,,,
6,,,,,
7,"$ I_{MSA} * sim_{DV}(pc_{i}, pc_{j})$",,,,
8,,,,,
9,"$ I_{MSA} * sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,,,


('norm', 'docvecs', 'FE', '1985-95')


,,,,,
0,,(1),(2),(3),(4)
1,"$I(MSA \, Match)$",0.0399***,-0.0177***,0.0434***,-0.0169***
2,,(0.0042),(0.0047),(0.0043),(0.0049)
3,"$ sim_{DV}(pc_{i}, pc_{j})$",0.2475***,,0.2408***,
4,,(0.0024),,(0.0026),
5,"$sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,0.1719***,,0.1698***
6,,,(0.0026),,(0.0033)
7,"$ I_{MSA} * sim_{DV}(pc_{i}, pc_{j})$",,,0.0254***,
8,,,,(0.0041),
9,"$ I_{MSA} * sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,,,0.0051


('norm', 'docvecs', 'FE', '1995-05')


...

,,,,,
0,,(1),(2),(3),(4)
1,"$I(MSA \, Match)$",0.0053***,-0.0131***,0.0042***,0.0066***
2,,(0.0009),(0.0009),(0.0010),(0.0012)
3,"$ sim_{LDA}(pc_{i}, pc_{j})$",0.2451***,,0.2443***,
4,,(0.0011),,(0.0013),
5,"$sim_{LDA}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,0.2877***,,0.3074***
6,,,(0.0014),,(0.0018)
7,"$ I_{MSA} * sim_{LDA}(pc_{i}, pc_{j})$",,,0.0032,
8,,,,(0.0025),
9,"$ I_{MSA} * sim_{LDA}(pc_{i,MSA_i}, pc_{j,MSA_...",,,,-0.0559***


('raw', 'ldavecs', 'No FE', '1995-05')


,,,,,
0,,(1),(2),(3),(4)
1,"$I(MSA \, Match)$",0.0119***,-0.0082***,0.0072***,0.0075***
2,,(0.0007),(0.0007),(0.0008),(0.0009)
3,"$ sim_{LDA}(pc_{i}, pc_{j})$",0.2593***,,0.2561***,
4,,(0.0009),,(0.0010),
5,"$sim_{LDA}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,0.3043***,,0.3182***
6,,,(0.0011),,(0.0013)
7,"$ I_{MSA} * sim_{LDA}(pc_{i}, pc_{j})$",,,0.0126***,
8,,,,(0.0020),
9,"$ I_{MSA} * sim_{LDA}(pc_{i,MSA_i}, pc_{j,MSA_...",,,,-0.0418***


('raw', 'ldavecs', 'No FE', '2005-15')


,,,,,
0,,(1),(2),(3),(4)
1,"$I(MSA \, Match)$",0.0081***,-0.0099***,0.0070***,0.0055***
2,,(0.0006),(0.0007),(0.0007),(0.0008)
3,"$ sim_{LDA}(pc_{i}, pc_{j})$",0.2693***,,0.2686***,
4,,(0.0007),,(0.0008),
5,"$sim_{LDA}(pc_{i,MSA_i}, pc_{j,MSA_j})$",,0.3038***,,0.3149***
6,,,(0.0008),,(0.0010)
7,"$ I_{MSA} * sim_{LDA}(pc_{i}, pc_{j})$",,,0.0024,
8,,,,(0.0016),
9,"$ I_{MSA} * sim_{LDA}(pc_{i,MSA_i}, pc_{j,MSA_...",,,,-0.0362***


In [81]:
def conv_latex(l, title, ncols = 8):
    l = ' '.join(l.split())
    l = l.replace("\\toprule \\\ \\midrule ",\
                  "\\toprule \multicolumn{"+str(ncols)+"}{c}{"+title+"} \\\ \hline ")
#     l = l.replace(" & &", " & ")
    l = l.replace(" & \\\ ", " \\\ ")
    l = l.replace("(4) \\", "(4) \\\ \midrule  ")
    return l

for k,v in tabp.items():
    print("\n"+str(k)+"\n"+"\n")
    ncols = len(v.columns)
    with pd.option_context("max_colwidth", 1000):
        print(conv_latex(v.to_latex(index=False,escape=False, column_format="lcccccccc"),
                        title = k[-1],
                        ncols = ncols))


('norm', 'docvecs', 'FE', '1975-85')


\begin{tabular}{lcccccccc} \toprule \multicolumn{5}{c}{1975-85} \\ \hline & (1) & (2) & (3) & (4) \\ \midrule  \ $I(MSA \, Match)$ & & & \\ & & & \\ $ sim_{DV}(pc_{i}, pc_{j})$ & & & \\ & & & \\ $sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$ & & & \\ & & & \\ $ I_{MSA} * sim_{DV}(pc_{i}, pc_{j})$ & & & \\ & & & \\ $ I_{MSA} * sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$ & & & \\ & & & \\ $N$ & & & \\ Adjusted $R^2$ & & & \\ Year FE & True & True & True & True \\ PC FE & True & True & True & True \\ \bottomrule \end{tabular}

('norm', 'docvecs', 'FE', '1985-95')


\begin{tabular}{lcccccccc} \toprule \multicolumn{5}{c}{1985-95} \\ \hline & (1) & (2) & (3) & (4) \\ \midrule  \ $I(MSA \, Match)$ & 0.0399*** & -0.0177*** & 0.0434*** & -0.0169*** \\ & (0.0042) & (0.0047) & (0.0043) & (0.0049) \\ $ sim_{DV}(pc_{i}, pc_{j})$ & 0.2475*** & & 0.2408*** \\ & (0.0024) & & (0.0026) \\ $sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$ & & 0.1719*** & & 0.1698*** \\ & & (0.0026) & & (0.